In [1]:
%load_ext autotime
import seaborn as sns
import pandas as pd
import numpy as np


from gscore.parsers import sqmass
from gscore.parsers.osw import OSWFile
from gscore.parsers import queries


time: 1.32 s (started: 2022-01-31 18:32:07 +01:00)


In [2]:
sqmass_file_path = "/home/aaron/projects/ghost/data/spike_in/chromatograms/AAS_P2009_167.sqMass"
osw_file_path = "/home/aaron/projects/ghost/data/spike_in/openswath/AAS_P2009_167.osw"
with sqmass.SqMassFile(sqmass_file_path) as sqmass_file:
    chromatograms = sqmass_file.parse()
with OSWFile(osw_file_path) as osw_file:
    precursors = osw_file.parse_to_precursors(
        query=queries.SelectPeakGroups.FETCH_CHROMATOGRAM_TRAINING_RECORDS
    )
precursors.set_chromatograms(chromatograms)

Cleaning unused score columns.
time: 2min 1s (started: 2022-01-31 18:32:10 +01:00)


In [3]:
labels = []
scores = []
chroms = []

for precursor in precursors:

    precursor.peakgroups.sort(key=lambda x: x.scores["PROBABILITY"], reverse=True)

    peakgroup = precursor.peakgroups[0]

    if peakgroup.chromatograms:

        score_array = peakgroup.get_sub_score_column_array(True)

        scores.append(np.array(score_array))

        labels.append(peakgroup.target)

        chroms.append(
            peakgroup.get_chromatogram_intensity_arrays(
                interpolated=True, use_relative_intensities=True
            )
        )


labels = np.array(labels).reshape(-1, 1)
scores = np.array(scores)
chroms = np.array(chroms).reshape((-1, 1, 6, 25))

time: 27.2 s (started: 2022-01-31 18:34:12 +01:00)


In [4]:
from gscore.models.deep_chrom_feature_classifier import DeepChromFeatureScorer

time: 224 µs (started: 2022-01-31 18:34:39 +01:00)


In [5]:
from gscore.scaler import Scaler

time: 300 µs (started: 2022-01-31 18:34:39 +01:00)


In [6]:
scaler = Scaler()

scores = scaler.fit_transform(scores)

time: 87.2 ms (started: 2022-01-31 18:34:39 +01:00)


In [7]:
scores.shape

(49062, 33)

time: 2.47 ms (started: 2022-01-31 18:34:39 +01:00)


In [8]:
model = DeepChromFeatureScorer(
    num_features=33,
    threads=10,
    max_epochs=5
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


time: 51.8 ms (started: 2022-01-31 18:34:39 +01:00)


In [9]:
model.fit(
    chroms,
    scores,
    labels
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | conv_layers   | Sequential | 10.9 K
1 | linear_layers | Sequential | 2.1 M 
---------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.317     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

time: 34.1 s (started: 2022-01-31 18:34:39 +01:00)


In [10]:
scaler.save("/home/aaron/projects/ghost/data/spike_in/deepchrom/dc_v_t_1_29.scaler")

time: 2.18 ms (started: 2022-01-31 18:35:13 +01:00)


In [11]:
model.save("/home/aaron/projects/ghost/data/spike_in/deepchrom/dc_v_t_1_29.model")

time: 53.4 ms (started: 2022-01-31 18:35:13 +01:00)


In [12]:
model.evaluate(
    chroms,
    scores,
    labels
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

/home/aaron/miniconda3/envs/ghostdev/lib/python3.8/site-packages/pytorch_lightning/loops/epoch/prediction_epoch_loop.py:172: UserWarning: Lightning couldn't infer the indices fetched for your dataloader.
  warning_cache.warn("Lightning couldn't infer the indices fetched for your dataloader.")


0.8308247552064687

time: 1.4 s (started: 2022-01-31 18:35:13 +01:00)


In [15]:
model = DeepChromFeatureScorer(
    num_features=33,
    threads=10,
    max_epochs=5
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


time: 16.5 ms (started: 2022-01-31 18:36:45 +01:00)


In [21]:
from gscore.models.deep_chrom_feature_classifier import DeepChromFeatureModel

test = DeepChromFeatureModel.load_from_checkpoint(
    checkpoint_path="/home/aaron/projects/ghost/data/spike_in/deepchrom/dc_v_t_1_29.model",
    num_features=33
)

time: 47.5 ms (started: 2022-01-31 18:38:31 +01:00)


In [16]:
test.load("/home/aaron/projects/ghost/data/spike_in/deepchrom/dc_v_t_1_29.model")

RuntimeError: Error(s) in loading state_dict for DeepChromFeatureModel:
	size mismatch for linear_layers.0.weight: copying a param with shape torch.Size([1008, 1041]) from checkpoint, the shape in current model is torch.Size([1008, 1008]).

time: 32.3 ms (started: 2022-01-31 18:36:46 +01:00)


In [20]:
scores = scaler.transform(scores)

time: 10.5 ms (started: 2022-01-31 16:37:42 +01:00)


In [21]:
predictions = test.score(
    chromatograms=chroms,
    scores=scores
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

time: 1.31 s (started: 2022-01-31 16:37:43 +01:00)
